## CSc 8830: Computer Vision - Assignment 1 


#### With the OAK-D camera, set up your application to show a RGB stream from the mono camera and a depth map stream from the stereo camera simultaneously. Make a note of what is the maximum frame rate and resolution achievable?

In [ ]:
# install depthai & opencv-python
!pip install depthai opencv-python

In [1]:
import time
import cv2 as cv
import numpy as np
import depthai as dai

# Parameters
RGB_RESOLUTION = dai.ColorCameraProperties.SensorResolution.THE_1080_P
DEPTH_RESOLUTION = dai.MonoCameraProperties.SensorResolution.THE_400_P
RESIZE_DIMENSIONS = (720, 480)  
EXTENDED_DISPARITY = False  
SUBPIXEL_DISPARITY = False  
LR_CHECK = True  

# Create pipeline
pipeline = dai.Pipeline()

# RGB Camera configuration
rgb_camera = pipeline.create(dai.node.ColorCamera)
rgb_camera.setResolution(RGB_RESOLUTION)
xout_rgb = pipeline.createXLinkOut()
xout_rgb.setStreamName("rgb")
rgb_camera.video.link(xout_rgb.input)

# Left Mono Camera configuration
left_camera = pipeline.createMonoCamera()
left_camera.setResolution(DEPTH_RESOLUTION)
left_camera.setBoardSocket(dai.CameraBoardSocket.LEFT)

# Right Mono Camera configuration
right_camera = pipeline.createMonoCamera()
right_camera.setResolution(DEPTH_RESOLUTION)
right_camera.setBoardSocket(dai.CameraBoardSocket.RIGHT)

# Stereo Depth configuration
stereo_depth = pipeline.createStereoDepth()
stereo_depth.setExtendedDisparity(EXTENDED_DISPARITY)
stereo_depth.setSubpixel(SUBPIXEL_DISPARITY)
stereo_depth.setLeftRightCheck(LR_CHECK)
left_camera.out.link(stereo_depth.left)
right_camera.out.link(stereo_depth.right)

# Output queues
disparity_output = pipeline.createXLinkOut()
disparity_output.setStreamName("disparity")
stereo_depth.disparity.link(disparity_output.input)

# Initialize variables for FPS calculation
prev_frame_time = 0  

# Pipeline execution
with dai.Device(pipeline) as device:
    disparity_queue = device.getOutputQueue(name="disparity", maxSize=4, blocking=False)
    rgb_queue = device.getOutputQueue(name="rgb", maxSize=4, blocking=False)
    
    while True:
        current_time = time.time()
        fps = 1 / (current_time - prev_frame_time) if prev_frame_time else 0
        prev_frame_time = current_time
        
        # Display RGB frame
        if rgb_queue and rgb_queue.has():
            rgb_frame = rgb_queue.get()
            resized_rgb_frame = cv.resize(rgb_frame.getCvFrame(), RESIZE_DIMENSIONS, interpolation=cv.INTER_AREA)
            cv.imshow("RGB Frame", resized_rgb_frame)
        
        # Display depth map
        if disparity_queue and disparity_queue.has():
            disparity_frame = disparity_queue.get().getFrame()
            if disparity_frame is not None:
                # Normalize for better visualization
                normalized_disparity = (disparity_frame * (255 / stereo_depth.initialConfig.getMaxDisparity())).astype(np.uint8)
                cv.imshow("Depth Map", normalized_disparity)

        print("FPS:", int(fps))  # Print FPS in the console
        
        key = cv.waitKey(1)  # Wait for key press
        if key == ord('q'):  # Break the loop if 'q' is pressed
            cv.destroyAllWindows()  # Close all OpenCV windows
            break


/var/folders/cv/flgh8s7960bc7q380pyn0c6m0000gn/T/ipykernel_83109/606186875.py:27: DeprecationWarning: LEFT is deprecated, use CAM_B or address camera by name  instead.
  left_camera.setBoardSocket(dai.CameraBoardSocket.LEFT)
/var/folders/cv/flgh8s7960bc7q380pyn0c6m0000gn/T/ipykernel_83109/606186875.py:32: DeprecationWarning: RIGHT is deprecated, use CAM_C or address camera by name  instead.
  right_camera.setBoardSocket(dai.CameraBoardSocket.RIGHT)


FPS: 0
FPS: 68
FPS: 79
FPS: 90
FPS: 79
FPS: 79
FPS: 79
FPS: 79
FPS: 78
FPS: 79
FPS: 92
FPS: 79
FPS: 88
FPS: 79
FPS: 79
FPS: 88
FPS: 79
FPS: 6
FPS: 11
FPS: 28
FPS: 82
FPS: 83
FPS: 38
FPS: 86
FPS: 75
FPS: 89
FPS: 85
FPS: 89
FPS: 71
FPS: 53
FPS: 89
FPS: 82
FPS: 33
FPS: 88
FPS: 76
FPS: 50
FPS: 89
FPS: 90
FPS: 89
FPS: 88
FPS: 88
FPS: 89
FPS: 46
FPS: 88
FPS: 95
FPS: 92
FPS: 89
FPS: 89
FPS: 88
FPS: 88
FPS: 39
FPS: 87
FPS: 88
FPS: 88
FPS: 96
FPS: 87
FPS: 89
FPS: 41
FPS: 88
FPS: 89
FPS: 91
FPS: 89
FPS: 90
FPS: 98
FPS: 61
FPS: 35
FPS: 90
FPS: 89
FPS: 94
FPS: 89
FPS: 32
FPS: 89
FPS: 90
FPS: 89
FPS: 89
FPS: 89
FPS: 89
FPS: 58
FPS: 32
FPS: 86
FPS: 86
FPS: 88
FPS: 87
FPS: 87
FPS: 42
FPS: 89
FPS: 86
FPS: 89
FPS: 89
FPS: 89
FPS: 60
FPS: 38
FPS: 86
FPS: 89
FPS: 86
FPS: 87


#### 1. Report the calibration matrix for the camera chosen and verify (using an example) the same. 
#### 2. Point the camera to a chessboard pattern or any known set of reference points that lie on the same plane. Capture a series of 10 images by changing the orientation of the camera in each iteration. Select any 1 image, and using the image formation pipeline equation, set up the linear equations in matrix form and solve for intrinsic and extrinsic parameters (extrinsic for that particular orientation). You will need to make measurements of the actual 3D world points, and mark pixel coordinates. Once you compute the Rotation matrix, you also need to compute the angles of rotation along each axis. Choose your order of rotation based on your experimentation setup. 


In [2]:
import time
import glob
import cv2 as cv
import numpy as np
import depthai as dai
from pathlib import Path

def capture_monochrome_images(src='right', delay=1000, num_imgs = 10):
    if src not in ['right', 'left']: 
        print(f"Accepted params are [left, right], but entered src: {src}.")
        return;
    
    # Start defining a pipeline
    pipeline = dai.Pipeline()

    # Define a source - mono (grayscale) camera
    cam = pipeline.createMonoCamera()
    if src == 'right':
        cam.setBoardSocket(dai.CameraBoardSocket.RIGHT)
    elif src == 'left':
        cam.setBoardSocket(dai.CameraBoardSocket.LEFT)
    else:
        print("Invalid source for monochrome camera")
        return

    cam.setResolution(dai.MonoCameraProperties.SensorResolution.THE_480_P)

    # Create output
    xout = pipeline.createXLinkOut()
    xout.setStreamName(src)
    cam.out.link(xout.input)

    # Connect and start the pipeline
    with dai.Device(pipeline, usb2Mode=True) as device:

        # Output queue will be used to get the grayscale frames from the output defined above
        q = device.getOutputQueue(name=src, maxSize=4, blocking=False)

        # Make sure the destination path is present before starting to store the examples
        Path(f"captured_images/{src}").mkdir(parents=True, exist_ok=True)

        for i in range(num_imgs):
            print(f"Capturing image {i+1} from {src} camera...")
            # Blocking call, will wait until a new data has arrived
            inSrc = q.get()  
            # Data is originally represented as a flat 1D array, it needs to be converted into HxW form
            frame = inSrc.getCvFrame()
            # Frame is transformed and ready to be shown
            cv.imshow(src, frame)

            cv.imwrite(f"captured_images/{src}/{int(time.time() * 10000)}.png", frame)
            print(f"Image {i+1} captured from {src} camera")
            cv.waitKey(delay)  

            cv.destroyAllWindows()            


def capture_color_images(delay=1000, num_imgs = 10):
    # Start defining a pipeline
    pipeline = dai.Pipeline()

    # Define a source - color camera
    cam = pipeline.createColorCamera()
    cam.setResolution(dai.ColorCameraProperties.SensorResolution.THE_1080_P)

    # Create RGB output
    xout = pipeline.createXLinkOut()
    xout.setStreamName("rgb")
    cam.video.link(xout.input)

    # Connect and start the pipeline
    with dai.Device(pipeline, usb2Mode=True) as device:

        # Output queue will be used to get the color frames from the output defined above
        q = device.getOutputQueue(name="rgb", maxSize=4, blocking=False)

        # Make sure the destination path is present before starting to store the examples
        Path(f"captured_images/rgb").mkdir(parents=True, exist_ok=True)

        for i in range(num_imgs):
            print(f"Capturing image {i+1} from RGB camera...")
            # Blocking call, will wait until a new data has arrived
            inSrc = q.get()  
            # Data is originally represented as a flat 1D array, it needs to be converted into HxW form
            frame = inSrc.getCvFrame()
            # Frame is transformed and ready to be shown
            imS = cv.resize(frame, (960, 540)) # Resize image
            cv.imshow("rgb", imS)

            cv.imwrite(f"captured_images/rgb/{int(time.time() * 10000)}.png", frame)
            print(f"Image {i+1} captured from RGB camera")
            cv.waitKey(delay)  

            cv.destroyAllWindows()


def caliberate_camera(images, src):
    if src not in ['right', 'left', 'rgb']: 
        print(f"Accepted params are - [left, right, rgb], but entered src: {src}")
        print("Enter Correct value for src!!")
        return;

    # Termination criteria
    criteria = (cv.TERM_CRITERIA_EPS + cv.TERM_CRITERIA_MAX_ITER, 30, 0.001)

    # Prepare object points
    objp = np.zeros((6*9, 3), np.float32)
    objp[:,:2] = np.mgrid[0:9, 0:6].T.reshape(-1, 2)

    # Arrays to store object points and image points from all the images
    objpoints = []  # 3D point in real world space
    imgpoints = []  # 2D points in image plane

    notFound = []

    for fname in images:
        img = cv.imread(fname)
        gray = cv.cvtColor(img, cv.COLOR_BGR2GRAY)

        # Finding chess board corners
        ret, corners = cv.findChessboardCorners(gray, (9, 6), None)

        # If found, add object points, image points (after refining them)
        if ret == True:
            objpoints.append(objp)
            corners2 = cv.cornerSubPix(gray, corners, (11, 11), (-1, -1), criteria)
            imgpoints.append(corners)

            # Draw and display the corners
            cv.drawChessboardCorners(img, (9, 6), corners2, ret)
            cv.imshow('img', img)

            # Saving displayed corners for future references
            cv.imwrite(f"{fname.split('.')[0]}_corners.png", img)
            cv.waitKey(1000)
            cv.destroyAllWindows()
        else:
            # If corners not found, store the file name in the notFound list
            notFound.append(fname)
            print(f"Corners not found for {fname}")

    cv.destroyAllWindows()

    # Remove the images whose corners weren't found from the main image list
    for i in notFound:
        images.remove(i)

    # Camera calibration
    ret, mtx, dist, rvecs, tvecs = cv.calibrateCamera(objpoints, imgpoints, gray.shape[::-1], None, None)

    for fname in images:
        img = cv.imread(fname)
        h,  w = img.shape[:2]
        newcameramtx, roi = cv.getOptimalNewCameraMatrix(mtx, dist, (w,h), 1, (w,h))

        # Undistort the image
        dst = cv.undistort(img, mtx, dist, None, newcameramtx)

        # Crop the image
        x, y, w, h = roi
        dst = dst[y:y+h, x:x+w]

        # Save the undistorted image
        cv.imwrite(f"{fname.split('.')[0]}_result.png", dst)

    # Save camera matrix and distortion coefficients for future use
    print("Saving camera matrix...")
    np.savetxt(f"captured_images/{src}/camera_matrix.txt", mtx)
    print("Saving distortion vector...")
    np.savetxt(f"captured_images/{src}/distortion_vector.txt", dist)
    print("Saving rotational vectors...")
    np.savetxt(f"captured_images/{src}/rotation_vectors.txt", np.array(rvecs).reshape(-1, 3))
    print("Saving translation vectors...")
    np.savetxt(f"captured_images/{src}/translation_vectors.txt", np.array(tvecs).reshape(-1, 3))
    print('Done!!')


# Capture images and calibrate cameras
print("Capturing monochrome images from right camera...")
capture_monochrome_images('right')
print("Capturing monochrome images from left camera...")
capture_monochrome_images('left')
print("Capturing color images from rgb camera...")
capture_color_images()

# Calibrate cameras
print("Calibrating cameras...")
caliberate_camera(glob.glob('captured_images/right/*.png'), 'right')
caliberate_camera(glob.glob('captured_images/left/*.png'), 'left')
caliberate_camera(glob.glob('captured_images/rgb/*.png'), 'rgb')

Capturing monochrome images from right camera...


/var/folders/cv/flgh8s7960bc7q380pyn0c6m0000gn/T/ipykernel_83109/2685450293.py:19: DeprecationWarning: RIGHT is deprecated, use CAM_C or address camera by name  instead.
  cam.setBoardSocket(dai.CameraBoardSocket.RIGHT)
/var/folders/cv/flgh8s7960bc7q380pyn0c6m0000gn/T/ipykernel_83109/2685450293.py:34: DeprecationWarning: Use constructor taking 'UsbSpeed' instead
  with dai.Device(pipeline, usb2Mode=True) as device:


Capturing image 1 from right camera...
Image 1 captured from right camera
Capturing image 2 from right camera...
Image 2 captured from right camera
Capturing image 3 from right camera...
Image 3 captured from right camera
Capturing image 4 from right camera...
Image 4 captured from right camera
Capturing image 5 from right camera...
Image 5 captured from right camera
Capturing image 6 from right camera...
Image 6 captured from right camera
Capturing image 7 from right camera...
Image 7 captured from right camera
Capturing image 8 from right camera...
Image 8 captured from right camera
Capturing image 9 from right camera...
Image 9 captured from right camera
Capturing image 10 from right camera...
Image 10 captured from right camera
Capturing monochrome images from left camera...


/var/folders/cv/flgh8s7960bc7q380pyn0c6m0000gn/T/ipykernel_83109/2685450293.py:21: DeprecationWarning: LEFT is deprecated, use CAM_B or address camera by name  instead.
  cam.setBoardSocket(dai.CameraBoardSocket.LEFT)


Capturing image 1 from left camera...
Image 1 captured from left camera
Capturing image 2 from left camera...
Image 2 captured from left camera
Capturing image 3 from left camera...
Image 3 captured from left camera
Capturing image 4 from left camera...
Image 4 captured from left camera
Capturing image 5 from left camera...
Image 5 captured from left camera
Capturing image 6 from left camera...
Image 6 captured from left camera
Capturing image 7 from left camera...
Image 7 captured from left camera
Capturing image 8 from left camera...
Image 8 captured from left camera
Capturing image 9 from left camera...
Image 9 captured from left camera
Capturing image 10 from left camera...
Image 10 captured from left camera
Capturing color images from rgb camera...


/var/folders/cv/flgh8s7960bc7q380pyn0c6m0000gn/T/ipykernel_83109/2685450293.py:72: DeprecationWarning: Use constructor taking 'UsbSpeed' instead
  with dai.Device(pipeline, usb2Mode=True) as device:


Capturing image 1 from RGB camera...
Image 1 captured from RGB camera
Capturing image 2 from RGB camera...
Image 2 captured from RGB camera
Capturing image 3 from RGB camera...
Image 3 captured from RGB camera
Capturing image 4 from RGB camera...
Image 4 captured from RGB camera
Capturing image 5 from RGB camera...
Image 5 captured from RGB camera
Capturing image 6 from RGB camera...
Image 6 captured from RGB camera
Capturing image 7 from RGB camera...
Image 7 captured from RGB camera
Capturing image 8 from RGB camera...
Image 8 captured from RGB camera
Capturing image 9 from RGB camera...
Image 9 captured from RGB camera
Capturing image 10 from RGB camera...
Image 10 captured from RGB camera
Calibrating cameras...
Corners not found for captured_images/right/17120874221513.png
Corners not found for captured_images/right/17120874293572.png
Corners not found for captured_images/right/17120929353299.png
Corners not found for captured_images/right/17120929332715.png
Corners not found for c

In [12]:
import cv2
import numpy as np

def load_camera_parameters(camera_matrix_file, distortion_vector_file):
    camera_params = []
    extrinsic_params = []
    with open(camera_matrix_file, 'r') as f:
        for line in f:
            camera_params.append([float(num) for num in line.split(' ')])
    with open(distortion_vector_file, 'r') as f:
        for line in f:
            extrinsic_params.append([float(num) for num in line.split(' ')])
    return np.array(camera_params), np.array(extrinsic_params)

def correct_distortion(image, camera_params, extrinsic_params):
    height, width = image.shape[:2]
    new_camera_params, roi = cv2.getOptimalNewCameraMatrix(camera_params, extrinsic_params, (width, height), 1, (width, height))
    undistorted = cv2.undistort(image, camera_params, extrinsic_params, None, new_camera_params)
    x, y, w, h = roi
    undistorted = undistorted[y:y+h, x:x+w]
    return undistorted

def calculate_error(corners, camera_params, object_distance, known_square_size_mm):
    ret, corners = cv2.findChessboardCorners(corners, (9, 6), None)
    focal_length_pixels = (camera_params[0, 0] + camera_params[1, 1]) / 2
    if ret:
        square_size_pixels = np.linalg.norm(corners[3] - corners[4])
        square_size_mm = (square_size_pixels / focal_length_pixels) * object_distance 
        print(f"Size of Square in mm: {square_size_mm}")
        error = abs(square_size_mm - known_square_size_mm)
        return error
    return None

input_image = cv2.imread('captured_images/right/17120874242142.png')
camera_params, extrinsic_params = load_camera_parameters('captured_images/right/camera_matrix.txt', 'captured_images/right/distortion_vector.txt')
undistorted_image = correct_distortion(input_image, camera_params, extrinsic_params)
gray_image = cv2.cvtColor(undistorted_image, cv2.COLOR_BGR2GRAY)
known_square_size_mm = 20  # Set the known size of the chessboard square in mm
object_distance = 300  # Set the distance of the object from the camera in mm
calibration_error = calculate_error(gray_image, camera_params, object_distance, known_square_size_mm)
if calibration_error is not None:
    print(f"Calibration Error: {calibration_error} mm")
else:
    print("Chessboard corners not found.")

Size of Square in mm: 21.92162150557055
Calibration Error: 1.921621505570549 mm


In [15]:
import cv2
import numpy as np
import math

def load_image(image_path):
    return cv2.imread(image_path)

def find_chessboard_corners(image):
    gray_img = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    ret, corners = cv2.findChessboardCorners(gray_img, (9, 6), None)
    if ret:
        refined_corners = cv2.cornerSubPix(gray_img, corners, (11, 11), (-1, -1), 
                                            (cv2.TERM_CRITERIA_EPS + cv2.TERM_CRITERIA_MAX_ITER, 30, 0.001))
        return refined_corners
    return None

def extract_image_points(corners):
    top_left_corner = corners[0].ravel()
    top_right_corner = corners[7].ravel()
    bottom_right_corner = corners[-1].ravel()
    bottom_left_corner = corners[-8].ravel()
    return np.array([top_left_corner, top_right_corner, bottom_right_corner, bottom_left_corner])

def solve_pnp(obj_points, img_points, camera_params, extrinsic_params):
    ret, rvecs, tvecs = cv2.solvePnP(obj_points, img_points, camera_params, extrinsic_params)
    return ret, rvecs, tvecs

def rotation_matrix_to_euler_angles(rotation_matrix):
    sy = math.sqrt(rotation_matrix[0, 0] * rotation_matrix[0, 0] + rotation_matrix[1, 0] * rotation_matrix[1, 0])
    singular = sy < 1e-6
    if not singular:
        x = math.atan2(rotation_matrix[2, 1], rotation_matrix[2, 2])
        y = math.atan2(-rotation_matrix[2, 0], sy)
        z = math.atan2(rotation_matrix[1, 0], rotation_matrix[0, 0])
    else:
        x = math.atan2(-rotation_matrix[1, 2], rotation_matrix[1, 1])
        y = math.atan2(-rotation_matrix[2, 0], sy)
        z = 0
    return np.array([x, y, z])

# Load the chessboard image
chessboard_img = load_image('captured_images/right/17120874242142.png')

# Find and refine the chessboard corners
corners = find_chessboard_corners(chessboard_img)

if corners is not None:
    # Draw the corners on the image
    cv2.drawChessboardCorners(chessboard_img, (9, 6), corners, True)
    cv2.imshow('Corners', chessboard_img)
    cv2.waitKey(5)
    cv2.destroyAllWindows()

    # Extract image points for PnP
    img_points = extract_image_points(corners)

    # Define 3D real-world points
    obj_points = np.array([[0, 0, 0], [216, 0, 0], [216, 162, 0], [0, 162, 0]], dtype='float32')

    # Solve PnP
    ret, rvecs, tvecs = solve_pnp(obj_points, img_points, camera_params, extrinsic_params)

    # Convert rotation vectors to rotation matrix
    rotation_matrix, _ = cv2.Rodrigues(rvecs)

    # Convert rotation matrix to euler angles
    euler_angles = np.degrees(rotation_matrix_to_euler_angles(rotation_matrix))

    # Print results
    print("Intrinsic Camera Matrix:")
    print(camera_params, "\n")
    print("Extrinsic Rotation Matrix:")
    print(rotation_matrix, "\n")
    print("Extrinsic Translation Vector:")
    print(tvecs, "\n")
    print("Rotation Angles across X, Y, Z axes (degrees):")
    print(euler_angles, "\n")
else:
    print("Chessboard corners not found.")


Intrinsic Camera Matrix:
[[855.53046232   0.         322.91164513]
 [  0.         846.05776973 257.14797989]
 [  0.           0.           1.        ]] 

Extrinsic Rotation Matrix:
[[ 0.91573981  0.11456659  0.38509104]
 [ 0.09173585  0.87354035 -0.47802907]
 [-0.39115872  0.47307691  0.7894258 ]] 

Extrinsic Translation Vector:
[[ -99.27374148]
 [-130.57869143]
 [ 513.37507446]] 

Rotation Angles across X, Y, Z axes (degrees):
[30.93286966 23.02661748  5.72062055] 



#### 3. Write a script to find the real world dimensions (e.g. diameter of a ball, side length of a cube) of an object using perspective projection equations. Validate using an experiment where you image an object using your camera from a specific distance (choose any distance but ensure you are able to measure it accurately) between the object and camera. 

In [3]:
import cv2 as cv
import depthai as dai
from pathlib import Path

def capture_color_images(delay=1000, num_images=10):
    pipeline = dai.Pipeline()
    color_cam = pipeline.createColorCamera()
    color_cam.setResolution(dai.ColorCameraProperties.SensorResolution.THE_1080_P)
    xout_rgb = pipeline.createXLinkOut()
    xout_rgb.setStreamName("rgb")
    color_cam.video.link(xout_rgb.input)

    with dai.Device(pipeline, usb2Mode=True) as device:
        rgb_queue = device.getOutputQueue(name="rgb", maxSize=4, blocking=False)
        Path(f"captured_images/object/").mkdir(parents=True, exist_ok=True)

        for i in range(num_images):
            print(f"Capturing image {i+1} from RGB camera...")
            rgb_data = rgb_queue.get()  
            rgb_frame = rgb_data.getCvFrame()
            resized_frame = cv.resize(rgb_frame, (960, 540))
            cv.imshow("rgb", resized_frame)   
            cv.imwrite(f"captured_images/object/circular_object_{i+1}.png", rgb_frame)
            print(f"Image {i+1} captured from RGB camera")
            cv.waitKey(delay)  

        cv.destroyAllWindows()

# Capture images using the color camera
capture_color_images()

/var/folders/cv/flgh8s7960bc7q380pyn0c6m0000gn/T/ipykernel_83109/2902555592.py:22: DeprecationWarning: Use constructor taking 'UsbSpeed' instead
  with dai.Device(pipeline, usb2Mode=True) as device:


Capturing image 1 from RGB camera...
Image 1 captured from RGB camera
Capturing image 2 from RGB camera...
Image 2 captured from RGB camera
Capturing image 3 from RGB camera...
Image 3 captured from RGB camera
Capturing image 4 from RGB camera...
Image 4 captured from RGB camera
Capturing image 5 from RGB camera...
Image 5 captured from RGB camera
Capturing image 6 from RGB camera...
Image 6 captured from RGB camera
Capturing image 7 from RGB camera...
Image 7 captured from RGB camera
Capturing image 8 from RGB camera...
Image 8 captured from RGB camera
Capturing image 9 from RGB camera...
Image 9 captured from RGB camera
Capturing image 10 from RGB camera...
Image 10 captured from RGB camera


In [9]:
import cv2
import numpy as np

def find_circle_properties(image_path):
    image = cv2.imread(image_path)
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    blurred = cv2.GaussianBlur(gray, (5, 5), 0)
    circles = cv2.HoughCircles(blurred, cv2.HOUGH_GRADIENT, dp=1, minDist=50, param1=100, param2=30, minRadius=10, maxRadius=250)

    if circles is not None:
        circles = circles[0, :]
        x, y, r = circles[0]
        center = (int(x), int(y))
        width = int(2 * r)
        height = int(2 * r)
        return center, width, height
    else:
        print("No circle detected in the image.")
        return None, None, None

camera_matrix = np.loadtxt('captured_images/left/camera_matrix.txt')

print("Camera Intrinsic Matrix: ", camera_matrix)

img_path = "captured_images/object/circular_object_10.png"
object_dist = 300

fx, fy = camera_matrix[0, 0], camera_matrix[1, 1]
Z = object_dist
(x, y), w, h = find_circle_properties(img_path)
bbox = (x, y, w, h)

print(f"fx: {fx}, fy: {fy}, Z: {Z}")
print(f"Center (x, y): {x}, {y}; Width (w): {w}; Height (h): {h}")

Camera Intrinsic Matrix:  [[1.10337649e+03 0.00000000e+00 2.24967357e+02]
 [0.00000000e+00 1.10188906e+03 2.85431664e+02]
 [0.00000000e+00 0.00000000e+00 1.00000000e+00]]
fx: 1103.3764896235605, fy: 1101.8890633756332, Z: 300
Center (x, y): 850, 461; Width (w): 415; Height (h): 415


In [10]:
import cv2
import math

def convert_milli_to_inch(x):
    return x / 254

def calculate_object_distance(image, bbox, fx, fy, Z):
    x, y, w, h = bbox

    Image_point1x = x
    Image_point1y = y
    Image_point2x = x + w
    Image_point2y = y + h

    cv2.line(image, (Image_point1x, Image_point1y-h//2), (Image_point1x, Image_point2y-h//2), (0, 0, 255), 8)

    Real_point1x = Z * (Image_point1x / fx)
    Real_point1y = Z * (Image_point1y / fy)
    Real_point2x = Z * (Image_point2x / fx)
    Real_point2y = Z * (Image_point2x / fy)

    print("Real World Co-ordinates: ")
    print("\t", Real_point1x)
    print("\t", Real_point1y)
    print("\t", Real_point2x)
    print("\t", Real_point2y)

    dist = math.sqrt((Real_point2y - Real_point1y) ** 2 + (Real_point2x - Real_point1x) ** 2)

    val = round(convert_milli_to_inch(dist) * 10, 2)

    cv2.putText(image, str(val) + " mm", (Image_point1x - 200, (y + h) // 2 + 5),
                cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2)
    cv2.imwrite("circilar_object.png", image)

    return val

image = cv2.imread(img_path)
distance = calculate_object_distance(image, bbox, fx, fy, Z)
print("\nDiameter of circular object is: {} mm".format(distance))

Real World Co-ordinates: 
	 231.10878507752008
	 125.51172762920292
	 343.9442507330152
	 344.40853676993856

Diameter of circular object is: 9.7 mm


: 